In [1]:
import ipywidgets as widgets
import pandas
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as colors
import IPython


################
# IMPORT DATA #
###############

df_01_gms = pandas.read_csv("test_01_gms_01.csv")
df_01_gms.columns = ['ftype', 'bin', 'proc', 'io_fr', 'k3_fr', 'exp_s', 'rec_s']

df_01_sem = pandas.read_csv("test_01_sem_01.csv")
df_01_sem.columns = ['ftype', 'bin', 'proc', 'io_fr', 'rec_s']

df_02_gms = pandas.read_csv("test_02_gms_01.csv")
df_02_gms.columns = ['ftype', 'bin', 'proc', 'io_fr', 'k3_fr', 'exp_s', 'rec_s']

df_02_sem = pandas.read_csv("test_02_sem_01.csv")
df_02_sem.columns = ['ftype', 'bin', 'proc', 'io_fr', 'rec_s']

##############################################################################################
# TEST 01:                                                                                   #
##############################################################################################
#    The exposure was fixed at 1.6s. This corresponds to 120 frames at 75 frames per second. #
#    120 frames was chosen as it has a large number of even divisors.                        #
#    Then images were recorded changing the number of frames written to disk.                #
#    For example all 120 frames were written to disk, then 60 frames were written to disk    #
#    (groups of 2 frames written to each frame on disk), then 40 frames (groups of 3 frames) #
#    and so on.                                                                              #
#                                                                                            #
#    Timings were measured both from Digital Micrograph using the SEMCCD Debug Mode as well  #
#    as from SerialEM (Running on the Gatan K3 PC) using debug output '1K'                   #
##############################################################################################
# TEST 01 - GMS TIMINGS: #
##########################
template = 'ggplot2'
colorways = [colors.colorbrewer.Blues,
             colors.colorbrewer.Blues_r,
             colors.colorbrewer.Greens,
             colors.colorbrewer.Greens_r,
             colors.colorbrewer.Reds,
             colors.colorbrewer.Reds_r,
             colors.colorbrewer.Purples,
             colors.colorbrewer.Purples_r]

ftype_01_gms_ = widgets.ToggleButtons(
    layout=widgets.Layout(),
    options=[('MRC', 'mrc'), ('TIFF', 'tif')],
    description='File Type:',
    disabled=False,
    button_style='',
    style=widgets.ToggleButtonsStyle(),
    tooltips=['Movies saved as MRC', 'Movies saved as TIFF'])

binn_01_gms_ = widgets.ToggleButtons(
    layout=widgets.Layout(),
    options=[('Counted', 'count'), ('Super Resolution', 'super')],
    description='Binning:',
    disabled=False,
    button_style='',
    style=widgets.ToggleButtonsStyle(),
    tooltips=['Movies hardware binned to physical pixel size (bin = 2)',
              'Movies saved at super resolution (bin = 1)'])

proc_01_gms_ = widgets.ToggleButtons(
    layout=widgets.Layout(),
    options=[('Gain Normalized', 'normed'), ('Unprocessed', 'unproc')],
    description='Processing:',
    disabled=False,
    button_style='',
    style=widgets.ToggleButtonsStyle(),
    tooltips=['Movies gain normalized', 'Movies not gain normalized'])

iofr_01_gms_ = widgets.Dropdown(
    layout=widgets.Layout(),
    options=['1', '2', '3', '4', '5', '6', '8', '10', '12', '15', '20', '24', '30', '40', '60', '120'],
    value='120',
    description='Frames:',
    disabled=False)

timing_01_gms_ui = widgets.HBox([ftype_01_gms_, binn_01_gms_, proc_01_gms_, iofr_01_gms_])

add_01_gms_button = widgets.Button(
    layout=widgets.Layout(),
    description='Add',
    tooltips='Add current selection to plot')

rem_01_gms_button = widgets.Button(
    layout=widgets.Layout(),
    description='Remove',
    tooltips='Remove current selection to plot')

reset_01_gms_button = widgets.Button(
    layout=widgets.Layout(),
    description='Reset',
    tooltips='Reset plot')

stddev_01_gms_button = widgets.Button(
    layout=widgets.Layout(),
    description='Show StdDev',
    tooltips='Show error bars of 1 s.d.')

colorways_idx = 0
test_01_gms_plots = []
test_01_gms_hash = {}
hash_idx = 0

for ftype in df_01_gms.ftype.unique():
    df_01_gms_ftype = df_01_gms.loc[df_01_gms['ftype'] == ftype]
    for binn in df_01_gms_ftype.bin.unique():
        df_01_gms_binn = df_01_gms_ftype.loc[df_01_gms_ftype['bin'] == binn]
        for proc in df_01_gms_binn.proc.unique():
            df_01_gms_proc = df_01_gms_binn.loc[df_01_gms_binn['proc'] == proc]
            color_idx = 0
            colorway = colors.n_colors(
                colorways[colorways_idx][2],
                colorways[colorways_idx][-3],
                df_01_gms_proc.io_fr.unique().size,
                colortype='rgb')
            colorways_idx = colorways_idx + 1
            for iofr in df_01_gms_proc.io_fr.unique():
                df_01_gms_iofr = df_01_gms_proc.loc[df_01_gms_proc['io_fr'] == iofr]
                x_dat = df_01_gms_iofr.k3_fr.unique()
                y_dat = [df_01_gms_iofr.loc[df_01_gms_iofr['k3_fr'] == xx].rec_s.mean() for xx in x_dat]
                y_err = [df_01_gms_iofr.loc[df_01_gms_iofr['k3_fr'] == xx].rec_s.std() for xx in x_dat]
                
                plots_key = '{}_{}_{}_{}'.format(ftype, binn, proc, iofr)
                test_01_gms_hash[plots_key] = hash_idx
                test_01_gms_plots.append(go.Scatter(
                    x = x_dat,
                    y = y_dat,
                    error_y=dict(
                        type='data',
                        array=y_err,
                        visible=False),
                    name = '{} {} {} {}'.format(ftype, binn, proc, iofr),
                    hoverinfo = 'name+y',
                    line_shape = 'linear',
                    mode='lines+markers',
                    line = dict(color = colorway[color_idx]),
                    visible = False))

                hash_idx = hash_idx + 1
                color_idx = color_idx + 1

test_01_gms_x_tickvals = list(range(0, 121, 5))
test_01_gms_x_ticktext = ['{:d}'.format(xx) for xx in test_01_gms_x_tickvals]
test_01_gms_y_tickvals = list(range(0, 21, 2))
test_01_gms_y_ticktext = ['{:d}'.format(yy) for yy in test_01_gms_y_tickvals]

test_01_gms_fig = go.FigureWidget(
    data = test_01_gms_plots[:],
    layout = dict(
        template = template,
        title = 'K3 GMS time for a 120 frame 1.6s exposure',
        xaxis_title = 'Frame index (a.u.)',
        xaxis = dict(
            tickmode = 'array',
            tickvals = test_01_gms_x_tickvals,
            ticktext = test_01_gms_x_ticktext),
        yaxis_title = 'Time (s)',
        yaxis = dict(
            tickmode = 'array',
            tickvals = test_01_gms_y_tickvals,
            ticktext = test_01_gms_y_ticktext)))

def add_01_gms_clicked(b):
    add_key = '{}_{}_{}_{}'.format(ftype_01_gms_.value, binn_01_gms_.value, proc_01_gms_.value, iofr_01_gms_.value)
    if add_key in test_01_gms_hash:
        add_idx = test_01_gms_hash[add_key]
        with test_01_gms_fig.batch_update():
            test_01_gms_fig.data[add_idx].visible = True
            
def rem_01_gms_clicked(b):
    rem_key = '{}_{}_{}_{}'.format(ftype_01_gms_.value, binn_01_gms_.value, proc_01_gms_.value, iofr_01_gms_.value)
    if rem_key in test_01_gms_hash:
        rem_idx = test_01_gms_hash[rem_key]
        with test_01_gms_fig.batch_update():
            test_01_gms_fig.data[rem_idx].visible = False

def reset_01_gms_clicked(b):
    with test_01_gms_fig.batch_update():
        for reset_idx in test_01_gms_hash.values():
            test_01_gms_fig.data[reset_idx].visible = False

def stddev_01_gms_clicked(b):
    with test_01_gms_fig.batch_update():
        for stddev_idx in test_01_gms_hash.values():
            test_01_gms_fig.data[stddev_idx].error_y['visible'] = not test_01_gms_fig.data[stddev_idx].error_y['visible']

add_01_gms_button.on_click(add_01_gms_clicked)
rem_01_gms_button.on_click(rem_01_gms_clicked)
reset_01_gms_button.on_click(reset_01_gms_clicked)
stddev_01_gms_button.on_click(stddev_01_gms_clicked)
buttons_01_gms = widgets.HBox([add_01_gms_button, rem_01_gms_button, stddev_01_gms_button, reset_01_gms_button])
figures_01_gms = widgets.VBox([test_01_gms_fig])
test_01_gms_label = widgets.Label(value='Test 01 GMS: ')

##########################
# TEST 01 - SEM TIMINGS: #
##########################
ftype_01_sem_ = widgets.ToggleButtons(
    layout=widgets.Layout(),
    options=[('MRC', 'mrc'), ('TIFF', 'tif')],
    description='File Type:',
    disabled=False,
    button_style='',
    style=widgets.ToggleButtonsStyle(),
    tooltips=['Movies saved as MRC', 'Movies saved as TIFF'])

binn_01_sem_ = widgets.ToggleButtons(
    layout=widgets.Layout(),
    options=[('Counted', 'count'), ('Super Resolution', 'super')],
    description='Binning:',
    disabled=False,
    button_style='',
    style=widgets.ToggleButtonsStyle(),
    tooltips=['Movies hardware binned to physical pixel size (bin = 2)',
              'Movies saved at super resolution (bin = 1)'])

proc_01_sem_ = widgets.ToggleButtons(
    layout=widgets.Layout(),
    options=[('Gain Normalized', 'normed'), ('Unprocessed', 'unproc')],
    description='Processing:',
    disabled=False,
    button_style='',
    style=widgets.ToggleButtonsStyle(),
    tooltips=['Movies gain normalized', 'Movies not gain normalized'])

iofr_01_sem_ = widgets.Dropdown(
    layout=widgets.Layout(),
    options=['1', '2', '3', '4', '5', '6', '8', '10', '12', '15', '20', '24', '30', '40', '60', '120'],
    value='120',
    description='Frames:',
    disabled=False)

timing_01_sem_ui = widgets.HBox([ftype_01_sem_, binn_01_sem_, proc_01_sem_, iofr_01_sem_])

add_01_sem_button = widgets.Button(
    layout=widgets.Layout(),
    description='Add',
    tooltips='Add current selection to plot')

rem_01_sem_button = widgets.Button(
    layout=widgets.Layout(),
    description='Remove',
    tooltips='Remove current selection to plot')

reset_01_sem_button = widgets.Button(
    layout=widgets.Layout(),
    description='Reset',
    tooltips='Reset plot')

stddev_01_sem_button = widgets.Button(
    layout=widgets.Layout(),
    description='Show StdDev',
    tooltips='Show error bars of 1 s.d.')

colorways_idx = 0
test_01_sem_fig = go.FigureWidget()
test_01_sem_hash = {}
hash_idx = 0

for ftype in df_01_sem.ftype.unique():
    df_01_sem_ftype = df_01_sem.loc[df_01_sem['ftype'] == ftype]
    for binn in df_01_sem_ftype.bin.unique():
        df_01_sem_binn = df_01_sem_ftype.loc[df_01_sem_ftype['bin'] == binn]
        for proc in df_01_sem_binn.proc.unique():
            df_01_sem_proc = df_01_sem_binn.loc[df_01_sem_binn['proc'] == proc]
            color_idx = 0
            colorway = colors.n_colors(
                colorways[colorways_idx][2],
                colorways[colorways_idx][-3],
                df_01_sem_proc.io_fr.unique().size,
                colortype='rgb')
            colorways_idx = colorways_idx + 1
            for iofr in df_01_sem_proc.io_fr.unique():
                df_01_sem_iofr = df_01_sem_proc.loc[df_01_sem_proc['io_fr'] == iofr]
                x_dat = ['{}'.format(iofr)]
                y_dat = [df_01_sem_iofr.rec_s.mean()]
                y_err = [df_01_sem_iofr.rec_s.std()]
            
                plots_key = '{}_{}_{}_{}'.format(ftype, binn, proc, iofr)
                test_01_sem_hash[plots_key] = hash_idx
                test_01_sem_fig.add_trace(go.Bar(
                    x = x_dat,
                    y = y_dat,
                    error_y=dict(
                        type='data',
                        array=y_err,
                        visible=False),
                    name = '{} {} {} {}'.format(ftype, binn, proc, iofr),
                    marker_color = colorway[color_idx],
                    visible = False))

                hash_idx = hash_idx + 1
                color_idx = color_idx + 1

test_01_sem_y_tickvals = list(range(0, 21, 2))
test_01_sem_y_ticktext = ['{:d}'.format(yy) for yy in test_01_sem_y_tickvals]

test_01_sem_fig.update_layout(
    template = template,
    title = 'K3 SerialEM time for a 120 frame 1.6s exposure',
    xaxis_title = 'Number of frames written (a.u.)',
    yaxis_title = 'Time (s)',
    yaxis = dict(
        tickmode = 'array',
        tickvals = test_01_sem_y_tickvals,
        ticktext = test_01_sem_y_ticktext),
    yaxis_tickangle=90,
    barmode='group',
    bargroupgap=0,
    bargap=0)

def add_01_sem_clicked(b):
    add_key = '{}_{}_{}_{}'.format(ftype_01_sem_.value, binn_01_sem_.value, proc_01_sem_.value, iofr_01_sem_.value)
    if add_key in test_01_sem_hash:
        add_idx = test_01_sem_hash[add_key]
        with test_01_sem_fig.batch_update():
            test_01_sem_fig.data[add_idx].visible = True
        test_01_sem_fig.update_layout(barmode='group', bargroupgap=0, bargap=0)
            
def rem_01_sem_clicked(b):
    rem_key = '{}_{}_{}_{}'.format(ftype_01_sem_.value, binn_01_sem_.value, proc_01_sem_.value, iofr_01_sem_.value)
    if rem_key in test_01_sem_hash:
        rem_idx = test_01_sem_hash[rem_key]
        with test_01_sem_fig.batch_update():
            test_01_sem_fig.data[rem_idx].visible = False

def reset_01_sem_clicked(b):
    with test_01_sem_fig.batch_update():
        for reset_idx in test_01_sem_hash.values():
            test_01_sem_fig.data[reset_idx].visible = False

def stddev_01_sem_clicked(b):
    with test_01_sem_fig.batch_update():
        for stddev_idx in test_01_sem_hash.values():
            test_01_sem_fig.data[stddev_idx].error_y['visible'] = not test_01_sem_fig.data[stddev_idx].error_y['visible']

add_01_sem_button.on_click(add_01_sem_clicked)
rem_01_sem_button.on_click(rem_01_sem_clicked)
reset_01_sem_button.on_click(reset_01_sem_clicked)
stddev_01_sem_button.on_click(stddev_01_sem_clicked)
buttons_01_sem = widgets.HBox([add_01_sem_button, rem_01_sem_button, stddev_01_sem_button, reset_01_sem_button])
figures_01_sem = widgets.VBox([test_01_sem_fig])
test_01_sem_label = widgets.Label(value='Test 01 SerialEM: ')

##############################################################################################
# TEST 02:                                                                                   #
##############################################################################################
#    The frame rate to disk was fixed at 75 frames per second. This corresponds to the max   #
#    capture rate of the K3. Then images were recorded changing the length of the exposure   #
#    capturing a single frame (0.0133s exposure) sequentially up to forty frames (0.53s)     #                                                                     #
#                                                                                            #
#    Timings were again measured both from Digital Micrograph using the SEMCCD Debug Mode    #
#    as well as from SerialEM (Running on the Gatan K3 PC) using debug output '1K'           #
##############################################################################################
# TEST 02 - GMS TIMINGS: #
##########################
ftype_02_gms_ = widgets.ToggleButtons(
    layout=widgets.Layout(),
    options=[('MRC', 'mrc'), ('TIFF', 'tif')],
    description='File Type:',
    disabled=False,
    button_style='',
    style=widgets.ToggleButtonsStyle(),
    tooltips=['Movies saved as MRC', 'Movies saved as TIFF'])

binn_02_gms_ = widgets.ToggleButtons(
    layout=widgets.Layout(),
    options=[('Counted', 'count'), ('Super Resolution', 'super')],
    description='Binning:',
    disabled=False,
    button_style='',
    style=widgets.ToggleButtonsStyle(),
    tooltips=['Movies hardware binned to physical pixel size (bin = 2)',
              'Movies saved at super resolution (bin = 1)'])

proc_02_gms_ = widgets.ToggleButtons(
    layout=widgets.Layout(),
    options=[('Gain Normalized', 'normed'), ('Unprocessed', 'unproc')],
    description='Processing:',
    disabled=False,
    button_style='',
    style=widgets.ToggleButtonsStyle(),
    tooltips=['Movies gain normalized', 'Movies not gain normalized'])

iofr_02_gms_ = widgets.BoundedIntText(
    value = 40,
    min = 1,
    max = 40,
    step = 1,
    description = 'Frames :',
    style = {'description_width' : '150px'},
    disabled = False
)

iofr_02_gms__ = widgets.IntSlider(
    value = 40,
    min = 1,
    max = 40,
    step = 1,
    description = '',
    disabled = False,
    continuous_update = True,
    orientation = 'horizontal',
    readout = False
)

iofr_02_gms_ui = widgets.HBox([iofr_02_gms_, iofr_02_gms__])
iofr_02_gms_link = widgets.jslink((iofr_02_gms_, 'value'),
                                  (iofr_02_gms__, 'value'))

timing_02_gms_ui = widgets.VBox([widgets.HBox([ftype_02_gms_, binn_02_gms_, proc_02_gms_]), iofr_02_gms_ui])

add_02_gms_button = widgets.Button(
    layout=widgets.Layout(),
    description='Add',
    tooltips='Add current selection to plot')

rem_02_gms_button = widgets.Button(
    layout=widgets.Layout(),
    description='Remove',
    tooltips='Remove current selection to plot')

reset_02_gms_button = widgets.Button(
    layout=widgets.Layout(),
    description='Reset',
    tooltips='Reset plot')

stddev_02_gms_button = widgets.Button(
    layout=widgets.Layout(),
    description='Show StdDev',
    tooltips='Show error bars of 1 s.d.')

colorways_idx = 0
test_02_gms_plots = []
test_02_gms_hash = {}
hash_idx = 0

for ftype in df_02_gms.ftype.unique():
    df_02_gms_ftype = df_02_gms.loc[df_02_gms['ftype'] == ftype]
    for binn in df_02_gms_ftype.bin.unique():
        df_02_gms_binn = df_02_gms_ftype.loc[df_02_gms_ftype['bin'] == binn]
        for proc in df_02_gms_binn.proc.unique():
            df_02_gms_proc = df_02_gms_binn.loc[df_02_gms_binn['proc'] == proc]
            color_idx = 0
            colorway = colors.n_colors(
                colorways[colorways_idx][2],
                colorways[colorways_idx][-3],
                df_02_gms_proc.io_fr.unique().size,
                colortype='rgb')
            colorways_idx = colorways_idx + 1
            for iofr in df_02_gms_proc.io_fr.unique():
                df_02_gms_iofr = df_02_gms_proc.loc[df_02_gms_proc['io_fr'] == iofr]
                x_dat = df_02_gms_iofr.k3_fr.unique()
                y_dat = [df_02_gms_iofr.loc[df_02_gms_iofr['k3_fr'] == xx].rec_s.mean() for xx in x_dat]
                y_err = [df_02_gms_iofr.loc[df_02_gms_iofr['k3_fr'] == xx].rec_s.std() for xx in x_dat]
                
                plots_key = '{}_{}_{}_{}'.format(ftype, binn, proc, iofr)
                test_02_gms_hash[plots_key] = hash_idx
                test_02_gms_plots.append(go.Scatter(
                    x = x_dat,
                    y = y_dat,
                    error_y=dict(
                        type='data',
                        array=y_err,
                        visible=False),
                    name = '{} {} {} {}'.format(ftype, binn, proc, iofr),
                    hoverinfo = 'name+y',
                    line_shape = 'linear',
                    mode='lines+markers',
                    line = dict(color = colorway[color_idx]),
                    visible = False))

                hash_idx = hash_idx + 1
                color_idx = color_idx + 1

test_02_gms_x_tickvals = list(range(0, 41, 5))
test_02_gms_x_ticktext = ['{:d}'.format(xx) for xx in test_02_gms_x_tickvals]
test_02_gms_y_tickvals = list(range(0, 21, 2))
test_02_gms_y_ticktext = ['{:d}'.format(yy) for yy in test_02_gms_y_tickvals]

test_02_gms_fig = go.FigureWidget(
    data = test_02_gms_plots[:],
    layout = dict(
        template = template,
        title = 'K3 GMS time at 75 frames per second',
        xaxis_title = 'Frame index (a.u.)',
        xaxis = dict(
            tickmode = 'array',
            tickvals = test_02_gms_x_tickvals,
            ticktext = test_02_gms_x_ticktext),
        yaxis_title = 'Time (s)',
        yaxis = dict(
            tickmode = 'array',
            tickvals = test_02_gms_y_tickvals,
            ticktext = test_02_gms_y_ticktext)))

def add_02_gms_clicked(b):
    add_key = '{}_{}_{}_{}'.format(ftype_02_gms_.value, binn_02_gms_.value, proc_02_gms_.value, iofr_02_gms_.value)
    if add_key in test_02_gms_hash:
        add_idx = test_02_gms_hash[add_key]
        with test_02_gms_fig.batch_update():
            test_02_gms_fig.data[add_idx].visible = True
            
def rem_02_gms_clicked(b):
    rem_key = '{}_{}_{}_{}'.format(ftype_02_gms_.value, binn_02_gms_.value, proc_02_gms_.value, iofr_02_gms_.value)
    if rem_key in test_02_gms_hash:
        rem_idx = test_02_gms_hash[rem_key]
        with test_02_gms_fig.batch_update():
            test_02_gms_fig.data[rem_idx].visible = False

def reset_02_gms_clicked(b):
    with test_02_gms_fig.batch_update():
        for reset_idx in test_02_gms_hash.values():
            test_02_gms_fig.data[reset_idx].visible = False

def stddev_02_gms_clicked(b):
    with test_02_gms_fig.batch_update():
        for stddev_idx in test_02_gms_hash.values():
            test_02_gms_fig.data[stddev_idx].error_y['visible'] = not test_02_gms_fig.data[stddev_idx].error_y['visible']

add_02_gms_button.on_click(add_02_gms_clicked)
rem_02_gms_button.on_click(rem_02_gms_clicked)
reset_02_gms_button.on_click(reset_02_gms_clicked)
stddev_02_gms_button.on_click(stddev_02_gms_clicked)
buttons_02_gms = widgets.HBox([add_02_gms_button, rem_02_gms_button, stddev_02_gms_button, reset_02_gms_button])
figures_02_gms = widgets.VBox([test_02_gms_fig])
test_02_gms_label = widgets.Label(value='Test 02 GMS: ')

##########################
# TEST 02 - SEM TIMINGS: #
##########################
ftype_02_sem_ = widgets.ToggleButtons(
    layout=widgets.Layout(),
    options=[('MRC', 'mrc'), ('TIFF', 'tif')],
    description='File Type:',
    disabled=False,
    button_style='',
    style=widgets.ToggleButtonsStyle(),
    tooltips=['Movies saved as MRC', 'Movies saved as TIFF'])

binn_02_sem_ = widgets.ToggleButtons(
    layout=widgets.Layout(),
    options=[('Counted', 'count'), ('Super Resolution', 'super')],
    description='Binning:',
    disabled=False,
    button_style='',
    style=widgets.ToggleButtonsStyle(),
    tooltips=['Movies hardware binned to physical pixel size (bin = 2)',
              'Movies saved at super resolution (bin = 1)'])

proc_02_sem_ = widgets.ToggleButtons(
    layout=widgets.Layout(),
    options=[('Gain Normalized', 'normed'), ('Unprocessed', 'unproc')],
    description='Processing:',
    disabled=False,
    button_style='',
    style=widgets.ToggleButtonsStyle(),
    tooltips=['Movies gain normalized', 'Movies not gain normalized'])

iofr_02_sem_ = widgets.BoundedIntText(
    value = 40,
    min = 1,
    max = 40,
    step = 1,
    description = 'Frames :',
    style = {'description_width' : '150px'},
    disabled = False
)

iofr_02_sem__ = widgets.IntSlider(
    value = 40,
    min = 1,
    max = 40,
    step = 1,
    description = '',
    disabled = False,
    continuous_update = True,
    orientation = 'horizontal',
    readout = False
)

iofr_02_sem_ui = widgets.HBox([iofr_02_sem_, iofr_02_sem__])
iofr_02_sem_link = widgets.jslink((iofr_02_sem_, 'value'),
                                  (iofr_02_sem__, 'value'))

timing_02_sem_ui = widgets.VBox([widgets.HBox([ftype_02_sem_, binn_02_sem_, proc_02_sem_]), iofr_02_sem_ui])


add_02_sem_button = widgets.Button(
    layout=widgets.Layout(),
    description='Add',
    tooltips='Add current selection to plot')

rem_02_sem_button = widgets.Button(
    layout=widgets.Layout(),
    description='Remove',
    tooltips='Remove current selection to plot')

reset_02_sem_button = widgets.Button(
    layout=widgets.Layout(),
    description='Reset',
    tooltips='Reset plot')

stddev_02_sem_button = widgets.Button(
    layout=widgets.Layout(),
    description='Show StdDev',
    tooltips='Show error bars of 1 s.d.')

colorways_idx = 0
test_02_sem_fig = go.FigureWidget()
test_02_sem_hash = {}
hash_idx = 0

for ftype in df_02_sem.ftype.unique():
    df_02_sem_ftype = df_02_sem.loc[df_02_sem['ftype'] == ftype]
    for binn in df_02_sem_ftype.bin.unique():
        df_02_sem_binn = df_02_sem_ftype.loc[df_02_sem_ftype['bin'] == binn]
        for proc in df_02_sem_binn.proc.unique():
            df_02_sem_proc = df_02_sem_binn.loc[df_02_sem_binn['proc'] == proc]
            color_idx = 0
            colorway = colors.n_colors(
                colorways[colorways_idx][2],
                colorways[colorways_idx][-3],
                df_02_sem_proc.io_fr.unique().size,
                colortype='rgb')
            colorways_idx = colorways_idx + 1
            for iofr in df_02_sem_proc.io_fr.unique():
                df_02_sem_iofr = df_02_sem_proc.loc[df_02_sem_proc['io_fr'] == iofr]
                x_dat = ['{}'.format(iofr)]
                y_dat = [df_02_sem_iofr.rec_s.mean()]
                y_err = [df_02_sem_iofr.rec_s.std()]
            
                plots_key = '{}_{}_{}_{}'.format(ftype, binn, proc, iofr)
                test_02_sem_hash[plots_key] = hash_idx
                test_02_sem_fig.add_trace(go.Bar(
                    x = x_dat,
                    y = y_dat,
                    error_y=dict(
                        type='data',
                        array=y_err,
                        visible=False),
                    name = '{} {} {} {}'.format(ftype, binn, proc, iofr),
                    marker_color = colorway[color_idx],
                    visible = False))

                hash_idx = hash_idx + 1
                color_idx = color_idx + 1

test_02_sem_y_tickvals = list(range(0, 21, 2))
test_02_sem_y_ticktext = ['{:d}'.format(yy) for yy in test_02_sem_y_tickvals]

test_02_sem_fig.update_layout(
    template = template,
    title = 'K3 SerialEM time for at 75 frames per second',
    xaxis_title = 'Number of frames written (a.u.)',
    yaxis_title = 'Time (s)',
    yaxis = dict(
        tickmode = 'array',
        tickvals = test_02_sem_y_tickvals,
        ticktext = test_02_sem_y_ticktext),
    yaxis_tickangle=90,
    barmode='group',
    bargroupgap=0,
    bargap=0)

def add_02_sem_clicked(b):
    add_key = '{}_{}_{}_{}'.format(ftype_02_sem_.value, binn_02_sem_.value, proc_02_sem_.value, iofr_02_sem_.value)
    if add_key in test_02_sem_hash:
        add_idx = test_02_sem_hash[add_key]
        with test_02_sem_fig.batch_update():
            test_02_sem_fig.data[add_idx].visible = True
        test_02_sem_fig.update_layout(barmode='group', bargroupgap=0, bargap=0)
            
def rem_02_sem_clicked(b):
    rem_key = '{}_{}_{}_{}'.format(ftype_02_sem_.value, binn_02_sem_.value, proc_02_sem_.value, iofr_02_sem_.value)
    if rem_key in test_02_sem_hash:
        rem_idx = test_02_sem_hash[rem_key]
        with test_02_sem_fig.batch_update():
            test_02_sem_fig.data[rem_idx].visible = False

def reset_02_sem_clicked(b):
    with test_02_sem_fig.batch_update():
        for reset_idx in test_02_sem_hash.values():
            test_02_sem_fig.data[reset_idx].visible = False

def stddev_02_sem_clicked(b):
    with test_02_sem_fig.batch_update():
        for stddev_idx in test_02_sem_hash.values():
            test_02_sem_fig.data[stddev_idx].error_y['visible'] = not test_02_sem_fig.data[stddev_idx].error_y['visible']

add_02_sem_button.on_click(add_02_sem_clicked)
rem_02_sem_button.on_click(rem_02_sem_clicked)
reset_02_sem_button.on_click(reset_02_sem_clicked)
stddev_02_sem_button.on_click(stddev_02_sem_clicked)
buttons_02_sem = widgets.HBox([add_02_sem_button, rem_02_sem_button, stddev_02_sem_button, reset_02_sem_button])
figures_02_sem = widgets.VBox([test_02_sem_fig])
test_02_sem_label = widgets.Label(value='Test 02 SerialEM: ')

IPython.display.display(test_01_gms_label, timing_01_gms_ui, buttons_01_gms, figures_01_gms,
                        test_01_sem_label, timing_01_sem_ui, buttons_01_sem, figures_01_sem,
                        test_02_gms_label, timing_02_gms_ui, buttons_02_gms, figures_02_gms,
                        test_02_sem_label, timing_02_sem_ui, buttons_02_sem, figures_02_sem)

Label(value='Test 01 GMS: ')

    'data': [{'error_y': {'array': [0.0018569533817705192, 0.13655069371490938,
…

Label(value='Test 01 SerialEM: ')

    'data': [{'error_y': {'array': [0.7740244137644811], 'type': 'data', 'visibl…

Label(value='Test 02 GMS: ')

    'data': [{'error_y': {'array': [0.0033333333333333175], 'type': 'data', 'vis…

Label(value='Test 02 SerialEM: ')

    'data': [{'error_y': {'array': [0.0570331287742289], 'type': 'data', 'visibl…